In [1]:
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import Audio
import pandas as pd
import numpy as np 
import os
import soundfile as sf
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, auc

In [2]:
def cut_audio_file(audio_dir_test,file,start,lenght):
# to get the audio segment and save it as segment_lenght_filename.mp3
#file = filename.mp3
# start = seconds
# lenght = seconds
    filename=os.path.join(audio_dir_test,file)
    y, sr = librosa.load(filename, sr=None)
    # Define start and end times
    end_sec = start + lenght  # 4 seconds after the start
    # Convert times to samples
    start_sample = int(start * sr)
    end_sample = int(end_sec * sr)
    # Extract the segment
    segment = y[start_sample:end_sample]
    # Save the segment
    out_filename = f'extracted_segment_{lenght}_{file}'
    sf.write(os.path.join(audio_dir_test,out_filename), segment, sr)
    

In [3]:
#get all features out of giver file 
def get_features(file_path, file_name):
    y, sr = librosa.load(file_path)
    iter_ = int(y.shape[0]/sr)
    features = []
    for i in range(iter_):
        y_seg = y[i*sr:(sr*i+sr)]
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_seg, sr=sr))
        rms = np.mean(librosa.feature.rms(y=y_seg))
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_seg, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_seg, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_seg, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y_seg))
        mfcc = np.mean(librosa.feature.mfcc(y=y_seg, sr=sr, n_mfcc=20),axis=1)
        features.append([chroma_stft,rms,spec_cent,spec_bw,rolloff,zcr,*mfcc]) # starred expressions for mfcc to get 20 columns instead of 20 row
        # b = np.concatenate((b,a),axis=1) # ajout filename apres?
    return features
    

In [4]:
data_set = pd.read_csv('Data/DATASET-balanced.csv')


In [12]:
def make_test_features_df(audio_dir,filename):
    file_features=get_features(os.path.join(audio_dir,filename),filename)
    feature_list = []
    for segment_features in file_features:
            feature_list.append(segment_features)

    features_df = pd.DataFrame(feature_list,columns = data_set.columns[0:-1])
    return features_df


In [6]:
# Start here:
# make sure it the right path
audio_dir_test='data/DEMONSTRATION/DEMONSTRATION'
files = os.listdir(audio_dir_test)
files


['Youssef.wav',
 'Youssef_Eric Cartman.mp3',
 '.DS_Store',
 'Youssef_2_Female.mp3',
 'extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Elise.m4a',
 'extracted_segment_100_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'extracted_segment_10_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Taylor Swift Talks Record.mp3',
 'Youssef_2.wav',
 'linus-to-musk-DEMO.mp3',
 'Morgan Freeman.mp3',
 'Youssef_2_Male Reggaeton.mp3',
 'linus-original-DEMO.mp3']

In [7]:
#select the file to cut using files[]
file_to_cut = files[4]
file_to_cut

'extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3'

In [8]:
# cut the audio, start in second, and the lenght in seconds
cut_audio_file(audio_dir_test,file_to_cut,89,10)


In [9]:
files = os.listdir(audio_dir_test)
files

['extracted_segment_10_extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Youssef.wav',
 'Youssef_Eric Cartman.mp3',
 '.DS_Store',
 'Youssef_2_Female.mp3',
 'extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Elise.m4a',
 'extracted_segment_100_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'extracted_segment_10_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Taylor Swift Talks Record.mp3',
 'Youssef_2.wav',
 'linus-to-musk-DEMO.mp3',
 'Morgan Freeman.mp3',
 'Youssef_2_Male Reggaeton.mp3',
 'linus-original-DEMO.mp3']

In [28]:
# select file to extract
file_toextract = files[4]
file_toextract

'Youssef_2_Female.mp3'

In [25]:
new_df = make_test_features_df(audio_dir_test,file_toextract)

/Users/youssefelmaaoua/.pyenv/versions/3.10.6/envs/Deep_fake_voice_recognition/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [26]:
new_df

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.579425,0.009091,3272.031691,2748.658893,6460.939719,0.109297,-584.452087,40.298458,-22.754082,-1.489093,...,-1.366650,-4.047152,-7.080334,-1.750380,1.068146,-7.576068,-1.051204,0.668266,-2.085448,-2.743236
1,0.483896,0.054477,3361.817266,2880.979088,6781.490811,0.080167,-359.543243,40.143150,-15.552848,-1.718714,...,-7.730168,-3.043950,-8.780021,2.206362,-6.292978,-10.289605,-1.452826,-2.502946,-9.700011,2.217101
2,0.473311,0.049601,3885.734309,2504.323935,6411.266535,0.266080,-222.903549,19.115871,-6.625169,41.368000,...,-11.320024,-6.886254,-10.958886,-2.802951,-3.047965,-12.652057,-2.414281,-5.561193,-12.614676,3.238885
3,0.805879,0.000031,4598.784587,3450.277271,8802.920255,0.012962,-908.758728,8.912713,8.545962,1.591202,...,1.404977,5.956839,2.954048,6.175857,0.924086,4.277379,2.753268,5.432143,2.812247,7.724409


In [16]:
file_toextract[:-4]

'Morgan Freeman'

In [27]:
new_df.to_csv(f'data/{file_toextract[:-4]}.csv',index=False)